# Convalution

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [0]:
conv = nn.Conv2d(1, 1, 11, stride = 4, padding = 0)

In [3]:
conv

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))

In [4]:
inputs = torch.Tensor(1, 1, 227, 227)
inputs.shape

torch.Size([1, 1, 227, 227])

In [5]:
out = conv(inputs)
out.shape

torch.Size([1, 1, 55, 55])

In [6]:
conv = nn.Conv2d(1, 1, 7, stride = 2, padding = 0)
inputs = torch.Tensor(1, 1, 64, 64)
out = conv(inputs)
out.shape

torch.Size([1, 1, 29, 29])

# CNN implementation

In [8]:
inputs = torch.Tensor(1, 1, 28, 28)
conv1 = nn.Conv2d(1, 5, 5)
pool = nn.MaxPool2d(2)
out = conv1(inputs)
out2 = pool(out)
print(out.size())
print(out2.size())

torch.Size([1, 5, 24, 24])
torch.Size([1, 5, 12, 12])


# CNN

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device == "cuda":
  torch.cuda.manual_seed_all(777)

In [0]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [12]:
# MNIST dataset
mnist_train = dsets.MNIST(root = "MNIST_data/", train=True, transform=transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root = "MNIST_data/", train = False, transform = transforms.ToTensor(), download=True)


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


In [0]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size=batch_size, shuffle = True, drop_last = True)

In [0]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.fc1 = nn.Linear(3 * 3 * 128, 625)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [39]:
print(device)

cuda


In [0]:
model = CNN().to(device)

In [41]:
value = (torch.Tensor(1, 1, 28, 28)).to(device)
print(model(value).shape)

torch.Size([1, 10])


In [0]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [45]:
# traning
total_batch = len(data_loader)
print("Learning Start")

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad() # 학습할 메소드 무조건 있어야함
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print("Epoch : {0} Cost = {1}".format(epoch + 1, avg_cost))
print("Learning Finished")

Learning Start
Epoch : 1 Cost = 0.004469671286642551
Epoch : 2 Cost = 0.005250072572380304
Epoch : 3 Cost = 0.004597033839672804
Epoch : 4 Cost = 0.003351207124069333
Epoch : 5 Cost = 0.004699636250734329
Epoch : 6 Cost = 0.005113480146974325
Epoch : 7 Cost = 0.003378754947334528
Epoch : 8 Cost = 0.0032854359596967697
Epoch : 9 Cost = 0.0044650910422205925
Epoch : 10 Cost = 0.0017113161738961935
Epoch : 11 Cost = 0.004630264826118946
Epoch : 12 Cost = 0.0030562409665435553
Epoch : 13 Cost = 0.0027849723119288683
Epoch : 14 Cost = 0.0024717010091990232
Epoch : 15 Cost = 0.0034224160481244326
Learning Finished


In [46]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()

  print("Accuracy : {:.3f}".format(accuracy.item()))

Accuracy : 0.985


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
